In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sbn

from sklearn.metrics import f1_score
from matplotlib import pyplot as plt

from other_paper.balancers  import BinaryBalancer, MulticlassBalancer
from other_paper import tools

from fmclp.main import fmclp
from sklearn.model_selection import train_test_split

from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

from fmclp.cuae_metric import cuae

### GPA_sex

In [2]:
gpa_sex = pd.read_csv('bar_pass_prediction.csv')

del gpa_sex['ID']
del gpa_sex['race1']
del gpa_sex['race2']
del gpa_sex['sex']
del gpa_sex['bar']
del gpa_sex['dnn_bar_pass_prediction']
del gpa_sex['pass_bar']
del gpa_sex['indxgrp2']
del gpa_sex['gender']
del gpa_sex['grad']
del gpa_sex['Dropout']
del gpa_sex['fulltime']
del gpa_sex['lsat']
del gpa_sex['zfygpa']
del gpa_sex['ugpa']
del gpa_sex['zgpa']
del gpa_sex['other']
del gpa_sex['asian']
del gpa_sex['black']
del gpa_sex['hisp']

def grouper_race(x):
    if x == 7:
        return 1
    else: 
        return 0
    
gpa_sex['race'] = gpa_sex['race'].apply(grouper_race)

def grouper_gpa(x):
    if x>3.4:
        return 2
    elif x <3.1:
        return 0
    else:
        return 1
gpa_sex = gpa_sex.dropna(how = 'any')

gpa_sex['gpa'] = gpa_sex['gpa'].apply(grouper_gpa)
gpa_sex['sex_race']= gpa_sex[['male','race']].apply(sum,axis=1).apply(lambda x: min(1,x))

del gpa_sex['male']
del gpa_sex['race']

del gpa_sex['bar1']
del gpa_sex['bar1_yr']
del gpa_sex['bar2']
del gpa_sex['bar2_yr']

gpa_sex = gpa_sex.rename(columns = {'gpa':'target', 'sex_race':'attr'})
gpa_sex = pd.get_dummies(gpa_sex,drop_first = True)

In [5]:
s=1
num=100
metric_list = []
accuracy_list = []
diff_list = []
ratio_list = []
variation_list = []

for i in range(num): 
    
    y = gpa_sex.drop(['target'], axis=1) 
    x = gpa_sex['target']
    y_train,y_test,x_train,x_test = train_test_split(y,x)
    gpa_sex_classif = LGBMClassifier()
    gpa_sex_classif.fit(y_train, x_train)
    gpa_sex_preds = gpa_sex_classif.predict(y_test)
    
    a = y_train['attr'].values
    y = x_train.values
    y_ = np.array(gpa_sex_classif.predict(y_train))

    gpa_sex_balancer = MulticlassBalancer(y=y, y_=y_, a=a)
    gpa_sex_balancer.adjust(goal='odds')
    gpa_sex_preds = gpa_sex_balancer.predict(y_ = np.array(gpa_sex_classif.predict(y_test)),
                                         a = y_test['attr'].values)
    
    accuracy = accuracy_score(gpa_sex_preds, x_test)
    metric = cuae(y_true = x_test, y_pred=gpa_sex_preds, sensitive_features=y_test['attr'].values)
    
    accuracy_list.append(accuracy)
    metric_list.append(metric)
    diff_list.append(metric['diff'])
    ratio_list.append(metric['ratio'])
    variation_list.append(metric['variation'])

    print(s)
    s+=1
    
accuracy_list = np.array(accuracy_list)
diff_list = np.array(diff_list)
ratio_list = np.array(ratio_list)
variation_list = np.array(variation_list)

accuracy_mean = accuracy_list.mean()
diff_mean = diff_list.mean()
ratio_mean = ratio_list.mean()
variation_mean = variation_list.mean()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [6]:
file = open("comparison_results/gpa_sex.txt", 'w') 
file.write(f"""accuracy: {accuracy_list}
diff: {diff_list}
ratio: {ratio_list}
var: {variation_list}

accuracy_mean: {accuracy_list.mean()}
diff_mean: {diff_list.mean()}
ratio_mean: {ratio_list.mean()}
var_mean: {variation_list.mean()}
""")
file.close()

In [7]:
print(f"""average_accuracy: {accuracy_list[:10].mean()}
average_diff: {diff_list[:10].mean()}
average_ratio: {ratio_list[:10].mean()}
average_variation: {variation_list[:10].mean()}""")

average_accuracy: 0.6589974544742511
average_diff: 0.16004216450016986
average_ratio: 1.3851627813031817
average_variation: 0.4000467774928098
